In [ ]:
from pymongo import MongoClient
import pymongo as pym
import seaborn as sns
from bson.objectid import ObjectId
from bson import Int64
%matplotlib inline

import matplotlib.pyplot as plt


from pylab import *
from scipy.optimize import curve_fit

import independent_cascade as ind_casc

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

from matplotlib.cbook import flatten

from scipy.optimize import minimize

import datetime
import pytz

from matplotlib.cbook import flatten

from scipy import stats

from sklearn import linear_model
from sklearn.metrics import f1_score, accuracy_score
import statistics

from sklearn import model_selection
import sklearn
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
# Connect to localhost instance (default)
client = MongoClient()
db = client.faken
coll = db.bias_tweets_unique
cursor = db.bias_tweets_unique.find()

In [ ]:
# Find tweets that have #FakeNews related keywords
cursor_fakenews = coll.find( {"$or" :  [{'text': {"$regex": '#fakenews', "$options": 'i' }},
                                {'text': {"$regex": 'fakenews', "$options": 'i' }},
                                {'text': {"$regex": 'fake-news', "$options": 'i' }},
                                {'text': {"$regex": '#fake-news', "$options": 'i' }},
                                {'text': {"$regex": 'posttruth', "$options": 'i' }},
                                {'text': {"$regex": '#post-truth', "$options": 'i' }},
                                {'text': {"$regex": 'alternativefact', "$options": 'i' }},
                                {'text': {"$regex": '#alternativefact', "$options": 'i' }},
                                {'text': {"$regex": 'alternative-fact', "$options": 'i' }},
                                {'text': {"$regex": '#alternative-fact', "$options": 'i' }} ]})

In [ ]:
# Same query as before as we need it for both DBs later: UIDs are different!
cursor_fakenews_innon = db.nonfakes.find( {"$or" :  [{'text': {"$regex": '#fakenews', "$options": 'i' }},
                                {'text': {"$regex": 'fakenews', "$options": 'i' }},
                                {'text': {"$regex": 'fake-news', "$options": 'i' }},
                                {'text': {"$regex": '#fake-news', "$options": 'i' }},
                                {'text': {"$regex": 'posttruth', "$options": 'i' }},
                                {'text': {"$regex": '#post-truth', "$options": 'i' }},
                                {'text': {"$regex": 'alternativefact', "$options": 'i' }},
                                {'text': {"$regex": '#alternativefact', "$options": 'i' }},
                                {'text': {"$regex": 'alternative-fact', "$options": 'i' }},
                                {'text': {"$regex": '#alternative-fact', "$options": 'i' }} ]})

In [ ]:
asd90() # Not to rerun
for eleme in cursor_fakenews_innon:
    db.nonfakes.delete_one({'_id': ObjectId(str(eleme['_id']))})

In [ ]:
asdafas() # Not rerun
for element in cursor_fakenews.rewind():
    db.fakes.insert_one(element)

In [ ]:
asdafasjdw() # Not to rerun
    print(db.nonfakes.count())
    for element in cursor_fakenews_innon.rewind():
        if db.nonfakes.delete_many({'_id': ObjectId(str(element['_id']))}).deleted_count != 0:
            print(1)

    print(db.nonfakes.count())

In [ ]:
# Retweet statistics
def times_timelist(theCursor):
    times = dict()
    timelist = list()
    for el in theCursor:
        try:
            timelist.append(el['retweet_reaction_time_sec'])
        except KeyError:
            continue
        try:
            times[el['retweet_reaction_time_sec']] += 1
        except KeyError:
            times[el['retweet_reaction_time_sec']] = 1
    return (times, timelist)

In [ ]:
times_timelist_computed_nonfakes = times_timelist(db.nonfakes.find())

In [ ]:
times_timelist_computed_fakes = times_timelist(db.fakes.find())

In [ ]:
max(times_timelist_computed_nonfakes[0])

In [ ]:
# Seaborn graphs
sns.set()
ax = sns.distplot(list(filter(lambda x: x<40000, times_timelist_computed_fakes[0])))

In [ ]:
sns.set()
ax = sns.distplot(list(filter(lambda x: x<1000000, times_timelist_computed_nonfakes[0])))

In [ ]:
# Fit to retweet delay data
x_fake = list()
y_fake = list()
for k in sorted(times_timelist_computed_fakes[0].keys()):
    if k<4000:
        x_fake.append(k)
        y_fake.append(times_timelist_computed_fakes[0][k])

x_fake = np.array(x_fake)
y_fake = np.array(y_fake)

def func(x, a, c, d):
    return a*np.exp(-c*x)+d

popt_f, pcov_f = curve_fit(func, x_fake, y_fake, p0=(45, 1e-12, 20))

In [ ]:
x_nfake = list()
y_nfake = list()
for k in sorted(times_timelist_computed_nonfakes[0].keys()):
    if k<1000000:
        x_nfake.append(k)
        y_nfake.append(times_timelist_computed_nonfakes[0][k])

x_nfake = np.array(x_nfake)
y_nfake = np.array(y_nfake)

def func(x, a, c, d):
    return a*np.exp(-c*x)+d

popt_nf, pcov_nf = curve_fit(func, x_nfake, y_nfake, p0=(45, 1e-12, 20))

In [ ]:
print(popt_nf)
print(popt_f)

In [ ]:
plot(xx_nf, np.array(list(map(lambda x: x/max(yy_nf),yy_nf))))# max > 8000
plot(xx_f,  np.array(list(map(lambda x: x/max(yy_f),yy_f)))) #max in 5

In [ ]:
# Functions to create retweet graphs and helper functions
def make_rt_graph_for_fakesres(cursor, max_seconds=False):
    graph = nx.DiGraph()
    graph_dict = dict()
    for smth in cursor:
        orig_id = ''
        graph_dict[str(smth['msg_id'])] = smth
        try: 
            orig_id = str(smth['retweeted_msg_id'])
            orig_id = orig_id.split('_')[-1]
            if max_seconds and smth['retweet_reaction_time_sec'] > max_seconds: # 5 minutes
                continue
        except KeyError: 
            continue
        graph.add_edges_from([(orig_id, str(smth['msg_id']))])
        
        el = smth
        try:
            supp=list(flatten(el["topics"]))
        except KeyError:
            continue
        if "TRUMP" in supp:
            el["TRUMP"]=1
        else:
            el["TRUMP"]=0
        if "HILARY" in supp:
            el["HILARY"]=1
        else:
            el["HILARY"]=0
        del el["topics"]

        db.res.insert_one(el)
    return (graph, graph_dict)

def make_rt_graph_for_nfakesres(cursor, max_seconds=False):
    graph = nx.DiGraph()
    graph_dict = dict()
    for smth in cursor:
        orig_id = ''
        graph_dict[str(smth['msg_id'])] = smth
        try: 
            orig_id = str(smth['retweeted_msg_id'])
            orig_id = orig_id.split('_')[-1]
            if max_seconds and smth['retweet_reaction_time_sec'] > max_seconds: # 5 minutes
                continue
        except KeyError: 
            continue
        graph.add_edges_from([(orig_id, str(smth['msg_id']))])
        
        el = smth
        try:
            supp=list(flatten(el["topics"]))
        except KeyError:
            continue
        if "TRUMP" in supp:
            el["TRUMP"]=1
        else:
            el["TRUMP"]=0
        if "HILARY" in supp:
            el["HILARY"]=1
        else:
            el["HILARY"]=0
        del el["topics"]

        db.fres.insert_one(el)
    return (graph, graph_dict)

def make_rt_graph_from_collection(collection, max_seconds=False):
    graph = nx.DiGraph()
    graph_dict = dict()
    cursor = ''
    if max_seconds:
        cursor = collection.find( { 'retweet_reaction_time_sec': {'$lt': int(max_seconds)}})
    else:
        cursor = collection.find()
    for smth in cursor:
        orig_id = ''
        graph_dict[str(smth['msg_id'])] = smth
        try: 
            orig_id = str(smth['retweeted_msg_id'])
            orig_id = orig_id.split('_')[-1]
        except KeyError: 
            continue
        graph.add_edges_from([(orig_id, str(smth['msg_id']))])

    return (graph, graph_dict)

In [ ]:
fake_graph = nx.DiGraph()
fake_graph_dict = dict()
(fake_graph, fake_graph_dict) = make_rt_graph_for_fakesres(db.fakes.find())

In [ ]:
def find_root(gra):
     return nx.topological_sort(gra)[0]

In [ ]:
def find_max_comp(graph):
    max_comp = 0
    max_size = 0
    for c in nx.weakly_connected_component_subgraphs(graph):
        if (c.number_of_nodes() > max_size):
            max_size = c.number_of_nodes()-1
            max_comp = c
    return (max_comp, max_size)

def find_all_components(graph):
    return list(nx.weakly_connected_component_subgraphs(graph))

In [ ]:
s_cn = 0
for gr in nx.weakly_connected_component_subgraphs(fake_graph):
    root = find_root(gr)
    s_cn += 1
    for node in gr:
        theEdges = list()
        if node == root:
            continue
        for e in max_comp.edges():
            if node in e:
                theEdges.append(e)

        if len(theEdges) > 1:
            print(theEdges)
            print(root)
            

print("Analyzed ", s_cn, " subgraphs")

In [ ]:
fig = plt.figure(figsize=(20, 18))
matplotlib.pyplot.subplot(121)
nx.draw(max_comp, with_labels=True, font_weight='bold')

In [ ]:
def find_reshare_order(subgr, gr_dic):
    time_msg_dict = dict()
    for element in subgr.nodes():
        try:
            t = gr_dic[element]['retweet_reaction_time_sec']
        except KeyError:
            t = 0 # father
        try:
            time_msg_dict[t].append(element)
        except KeyError:
            time_msg_dict[t] = [element]
    
    ret = list()gr_dic[element]
    for k in sorted(time_msg_dict.keys()):
        ret.append((k,time_msg_dict[k]))
    return ret

In [ ]:
#find_reshare_order(max_comp)

In [ ]:
list(fake_graph_dict.keys())[0]
'802523214995931136' in fake_graph_dict.keys()

In [ ]:
#####################
# Temporal Features #

def time_i(r_order, i): 
    try:
        return r_order[i][0]
    except IndexError:
        return 0

def time_first_half(r_order, k): 
    r_order = list(filter(lambda x: x[0] > 0, r_order)) # Do not count the 0-secs reshare of root
    max_index = int(k/2)-1
    lower_half_times = list()
    for x in r_order[0:max_index]:
        lower_half_times.append(x[0])
    deltalow = list()
    for i in range(len(lower_half_times)-1):
        deltalow.append(lower_half_times[i+1]-lower_half_times[i])
    if len(deltalow) > 0:
        return sum(deltalow)/int(len(deltalow))
    else:
        return 0

def time_second_half(r_order, k):
    r_order = list(filter(lambda x: x[0] > 0, r_order)) # Do not count the 0-secs reshare of root
    min_index = int(k/2)-1
    top_half_times = list()
    for x in r_order[min_index:int(k)-1]:
        if x[0] > 0: # Do not count the 0-secs reshare of root
            top_half_times.append(x[0])
    deltatop = list()
    for i in range(len(top_half_times)-1):
        deltatop.append(top_half_times[i+1]-top_half_times[i])
        
    if len(deltatop) > 0:
        return sum(deltatop)/int(len(deltatop))
    else:
        return 0
    

def time_change_reshares(r_order, k):
    or_order = list(r_order)#list(filter(lambda x: x[0] > 0, r_order))
    def f(x):
        res = 0
        t1 = 0
        tp2 = 0
        for i in range(or_order != [] and len(or_order)-1):
            t1 = time_i(or_order, i)
            tp1 = time_i(or_order, i+1) or t1
            res += ((tp1-t1)-(1+i)*x)**2
        return res
    x0 = 0.01
    return minimize(fun=f, x0=x0).x[0]

In [ ]:
res_order = find_reshare_order(max_comp)
max_k=len(res_order) -1
print(time_first_half(res_order, max_k))
print(time_second_half(res_order, max_k))
print(time_change_reshares(r_order, max_k))

In [ ]:
# Simple conversion
def time_str_to_datetime(time_str):
    return datetime.datetime.strptime(time_str, '%a %b %d %H:%M:%S %z %Y')

In [ ]:
r_order = find_reshare_order(max_comp)

In [ ]:
# Double check date format
db.fakes.find_one()['datetime']

In [ ]:
epoch = pytz.utc.localize(datetime.datetime.utcfromtimestamp(0))
times_dt = list(map(lambda x: time_str_to_datetime(x['datetime']), db.fakes.find({}, {'datetime':1, '_id':0})))
times = list(map(lambda x: (time_str_to_datetime(x['datetime'])-epoch).total_seconds(), db.fakes.find({}, {'datetime':1, '_id':0})))
t_min = min(times)
times = list(map(lambda x: x-t_min, times))

In [ ]:
##########
# For non fakes #

asdasdasdrg()
date_dict_nonfakes = dict()
for date in db.nonfakes.find({}, {'datetime':1, '_id':0}):
    try: 
        date_dict_nonfakes[time_str_to_datetime(date['datetime'])] += 1
    except KeyError:
        date_dict_nonfakes[time_str_to_datetime(date['datetime'])] = 1
date_dts_nf = list(date_dict_nonfakes.keys())
values_nf = list()
for d in date_dts_nf:
    values.append(date_dict_nonfakes[d])

dates_nf = matplotlib.dates.date2num(date_dts_nf)
matplotlib.pyplot.plot_date(dates_nf, values_nf)

In [ ]:
db.nonfakes.create_index([('datetime', pym.ASCENDING)])

In [ ]:
##########
# For non fakes #

date_dict_nonfakes = dict()
c=0
for date in db.nonfakes.find({}, {'datetime':1, '_id':0}):
    c+=1
    try: 
        date_dict_nonfakes[time_str_to_datetime(date['datetime'])] += 1
    except KeyError:
        date_dict_nonfakes[time_str_to_datetime(date['datetime'])] = 1
print(c)
print('done for')
date_dts_nf = list(date_dict_nonfakes.keys())
values_nf = list()
print('start appending')
for d in date_dts_nf:
    values_nf.append(date_dict_nonfakes[d])

dates_nf = matplotlib.dates.date2num(date_dts_nf)
matplotlib.pyplot.plot_date(dates_nf, values_nf)

In [ ]:
date_dict = dict()
for date in db.fakes.find({}, {'datetime':1, '_id':0}):
    try: 
        date_dict[time_str_to_datetime(date['datetime'])] += 1
    except KeyError:
        date_dict[time_str_to_datetime(date['datetime'])] = 1
date_dts = list(date_dict.keys())
values = list()
for d in date_dts:
    values.append(date_dict[d])
dates = matplotlib.dates.date2num(date_dts)
matplotlib.pyplot.plot_date(dates, values)

In [ ]:
date_dict = dict()
for date in db.bias_tweets_unique.find({}, {'datetime':1, '_id':0}):
    try: 
        date_dict[time_str_to_datetime(date['datetime'])] += 1
    except KeyError:
        date_dict[time_str_to_datetime(date['datetime'])] = 1
date_dts = list(date_dict.keys())
values = list()
for d in date_dts:
    values.append(date_dict[d])
dates = matplotlib.dates.date2num(date_dts)
matplotlib.pyplot.plot_date(dates, values)

In [ ]:
def component_percent_rt_after_k(comp, k):
    order = find_reshare_order(comp)
    count = 0
    for x in order:
        if x[0] > k:
            break
        else:
            count+=1
    return count/len(order)

def total_percent_rt_after_k(graph, k):
    tot = 0
    comps = find_all_components(graph)
    for subg in comps:
        tot += component_percent_rt_after_k(subg, k)
    return tot/len(comps)

In [ ]:
print(total_percent_rt_after_k(make_rt_graph(db.fakes.find(), 60)[0], 30)*100, '%', sep='')

In [ ]:
nf_graph = make_rt_graph_from_collection(db.nonfakes, 60)

In [ ]:
asdasddswef()
current_elements = list()
res_graph = nx.DiGraph()
max_len = 100000
for c,el in enumerate(db.nonfakes.find()):
    current_elements.append(el)
    if(c and c%max_len == 0):    
        res_graph = nx.compose(res_graph, make_rt_graph(current_elements)[0])
        current_elements = list()
res_graph = nx.compose(res_graph, make_rt_graph(current_elements)[0])
print(len(res_graph.edges()))

In [ ]:
print(total_percent_rt_after_k(nf_graph[0], 10)*100, '%', sep='')

In [ ]:
print(len(res_graph.edges()))
print(len(res_graph.nodes()))

In [ ]:
#find_reshare_order(find_max_comp(nf_graph[0])[0])
print(0%90)

In [ ]:
asdasdasd()

percentages = list()
for i in range(20):
    nf_graph = make_rt_graph(db.nonfakes.aggregate([ { '$sample': { 'size': 100000 } } ]))
    percentages.append(total_percent_rt_after_k(nf_graph[0], 10))

In [ ]:
sum(percentages)/len(percentages)

In [ ]:
total_percent_rt_after_k(res_graph, 10)

In [ ]:
max_comp_4m=find_max_comp(res_graph)

In [ ]:
r_ord_4m = find_reshare_order(max_comp_4m[0])

In [ ]:
max_comp_4m

In [ ]:
for node in max_comp_4m[0].nodes():
    print(node)
    break

In [ ]:
tutti_figli = list()
max_iter = 40000
print(datetime.datetime.now())
for c,ele in enumerate(db.nonfakes.find({'retweet_reaction_time_sec': {'$gte':0}}).limit(max_iter)):
    tutti_figli.append(ele)

print(datetime.datetime.now())

In [ ]:
len(tutti_figli)

In [ ]:
father_son = dict()
for f in tutti_figli:
    try:
        father_son[f['retweeted_msg_id']].append(f)
    except KeyError:
        father_son[f['retweeted_msg_id']] = [f]

In [ ]:
biggest_family = max(map(len,father_son.values()))
smallest_family = min(map(len,father_son.values()))

In [ ]:
print('Biggest family: ', biggest_family)
print('Smallest family: ',smallest_family)
print('Average family size: ', sum(list(map(len,father_son.values())))/len(father_son.values()))
print('Families: ', len(father_son.keys()))

In [ ]:
import copy

copia_figli =  copy.deepcopy(tutti_figli)

In [ ]:
#db.res.delete_many({})

In [ ]:
print(datetime.datetime.now())
fake_1000_makert = make_rt_graph(tutti_figli)
print(datetime.datetime.now())

In [ ]:
nfake_graph = nx.DiGraph()
nfake_graph_dict = dict()
(nfake_graph, nfake_graph_dict) = fake_1000_makert

In [ ]:
#max_comp=find_max_comp(fake_graph)[0]
c = 0
chg = list()
for comp in (nx.weakly_connected_component_subgraphs(nfake_graph)):
    res_order = find_reshare_order(comp, nfake_graph_dict)
    max_k= min(5, len(res_order[0][1]) - 1)
    tc = time_change_reshares(res_order, max_k)
    first = time_first_half(res_order, max_k)
    second = time_second_half(res_order, max_k)
    i_time = time_i(res_order, max_k)
    for node in comp.nodes():
        db.res.update_one({'msg_id':Int64(str(node))}, {'$set':
                                                {   'tmp_feat_k':max_k,
                                                    'tmp_feat_tk':i_time, 
                                                    'tmp_feat_firstk':first,
                                                    'tmp_feat_secondk':second,
                                                    'tmp_feat_time_chg':tc
                                                }
                                               })
        
    chg.append(tc)

In [ ]:
set(chg)

In [ ]:
all_avg = list()
for comp in (nx.weakly_connected_component_subgraphs(nfake_graph)):
    res_order = find_reshare_order(comp, nfake_graph_dict)
    media = 0
    mm  = map(lambda x: [x[0]]*len(x[1]), res_order)
    ll = list(mm)
    avg = sum((flatten(ll)))/len(ll)
    all_avg.append((len(ll), avg))

In [ ]:
tot_t = 0
tot_cn = 0
min_t = 100000000000000000000000000000000000
max_t = 0
for w,a in all_avg:
    tot_t += w*a
    tot_cn += w
    if a < min_t and a > 0:
        min_t = a
    if a > max_t:
        max_t = a

In [ ]:
flattened_all = list(flatten(map(lambda x: x[0]*[x[1]], all_avg)))
#nfake_graph.

In [ ]:
dummy = nx.Graph()
dummy.add_edges_from([('0','1'), ('0','2'), ('0','3'), ('0', '4'), ('0', '5')])

#dummy = nx.complete_graph(10)

nx.wiener_index(dummy)

In [ ]:
print(stats.mode(flattened_all))
stats.describe(flattened_all)

In [ ]:
nfwiener_total = dict()

for comp in (nx.weakly_connected_component_subgraphs(nfake_graph)):
    undir = nx.Graph()
    undir.add_edges_from(comp.edges())
    f_wien = nx.wiener_index(undir)
    for node in comp.nodes():
        nfwiener_total[node] = f_wien

In [ ]:
def prova(el,string):
    try:
        return el[string]
    except KeyError:
        return 0

In [ ]:
index =  db.res.create_index([('msg_id', pym.ASCENDING)], unique=True)

In [ ]:
(wiener_total.values)

In [ ]:
wiener_median = statistics.median(nfwiener_total.values())
c = 0
for node in nfwiener_total.keys():
    is_gt = int(nfwiener_total[node]>wiener_median)
    db.res.update_one({'msg_id':Int64(str(node))}, {'$set':
                                                {   'wiener_gt_median':is_gt,
                                                }
                                               })

In [ ]:
def compute_x_list_component(el):
    return [
                prova(el,'is_manual_reply'), 
                prova(el,'is_manual_retweet'),
                prova(el,'is_quote_retweet'), 
                prova(el,'is_reply_button'), 
                prova(el,'is_retweet_button'),
                prova(el,'TRUMP'), 
                prova(el,'HILARY'), 
                prova(el,'is_retweeted_user_verified'), 
                prova(el,'retweet_reaction_time_sec'),  
                prova(el,'retweeted_user_followers_count'),
                prova(el,'tmp_feat_k'),
                prova(el,'tmp_feat_tk'),
                prova(el,'tmp_feat_firstk'),
                prova(el,'tmp_feat_secondk'),
                prova(el,'tmp_feat_time_chg'),
             ]

In [ ]:
X=list()
Y=list()

for el in db.res.find():
    Y.append(prova(el,'wiener_gt_median'))
    X.append(compute_x_list_component(el))

logreg = linear_model.LogisticRegression(n_jobs=4)
logreg.fit(X[10000:], Y[10000:])

In [ ]:
x_test = X[:10000]
y_test = Y[:10000]
y_pred = logreg.predict(np.array(x_test))

In [ ]:
print('Acc', accuracy_score(y_pred=y_pred, y_true=y_test))
print('F1', f1_score(y_pred=y_pred, y_true=y_test))

In [ ]:
ftutti_figli = list()
print(datetime.datetime.now())
for c,ele in enumerate(db.fakes.find()):
    ftutti_figli.append(ele)

print(datetime.datetime.now())

In [ ]:
ffather_son = dict()
for f in ftutti_figli:
    try:
        ffather_son[f['retweeted_msg_id']].append(f)
    except KeyError:
        try:
            ffather_son[f['retweeted_msg_id']] = [f]
        except KeyError:
            ffather_son[f['msg_id']] = [f]

In [ ]:
fbiggest_family = max(map(len,ffather_son.values()))
fsmallest_family = min(map(len,ffather_son.values()))
print('Biggest family: ', fbiggest_family)
print('Smallest family: ',fsmallest_family)
print('Average family size: ', sum(list(map(len,ffather_son.values())))/len(ffather_son.values()))
print('Families: ', len(ffather_son.keys()))

In [ ]:
#db.fres.delete_many({})

In [ ]:
print(datetime.datetime.now())
nfake_1000_makert = make_rt_graph_for_fakesres(ftutti_figli)
print(datetime.datetime.now())

In [ ]:
fake_graph = nx.DiGraph()
fake_graph_dict = dict()
(fake_graph, fake_graph_dict) = nfake_1000_makert

In [ ]:
#max_comp=find_max_comp(fake_graph)[0]
c = 0
chg = list()
for comp in (nx.weakly_connected_component_subgraphs(fake_graph)):
    res_order = find_reshare_order(comp, fake_graph_dict)
    max_k= min(5, len(res_order[0][1]) - 1)
    tc = time_change_reshares(res_order, max_k)
    first = time_first_half(res_order, max_k)
    second = time_second_half(res_order, max_k)
    i_time = time_i(res_order, max_k)
    for node in comp.nodes():
        db.fres.update_one({'msg_id':Int64(str(node))}, {'$set':
                                                {   'tmp_feat_k':max_k,
                                                    'tmp_feat_tk':i_time, 
                                                    'tmp_feat_firstk':first,
                                                    'tmp_feat_secondk':second,
                                                    'tmp_feat_time_chg':tc
                                                }
                                               })
        
    chg.append(tc)

In [ ]:
set(chg)

In [ ]:
fwiener_total = dict()

for comp in (nx.weakly_connected_component_subgraphs(fake_graph)):
    undir = nx.Graph()
    undir.add_edges_from(comp.edges())
    f_wien = nx.wiener_index(undir)
    for node in comp.nodes():
        fwiener_total[node] = f_wien

In [ ]:
def prova(el,string):
    try:
        return el[string]
    except KeyError:
        return 0

In [ ]:
index =  db.fres.create_index([('msg_id', pym.ASCENDING)], unique=True)

In [ ]:
wiener_median = statistics.median(fwiener_total.values())
c = 0
for node in fwiener_total.keys():
    is_gt = int(fwiener_total[node]>wiener_median)
    db.fres.update_one({'msg_id':Int64(str(node))}, {'$set':
                                                {   'wiener_gt_median':is_gt,
                                                }
                                               })

In [ ]:
fX=list()
fY=list()

for el in db.fres.find():
    fY.append(prova(el,'wiener_gt_median'))
    fX.append(compute_x_list_component(el))

fake_logreg = linear_model.LogisticRegression(n_jobs=4)
fake_logreg.fit(fX[4000:], fY[4000:])

In [ ]:
x_test = fX[:4000]
y_test = fY[:4000]
y_pred = fake_logreg.predict(np.array(x_test))

print('Acc', accuracy_score(y_pred=y_pred, y_true=y_test))
print('F1', f1_score(y_pred=y_pred, y_true=y_test))

In [ ]:
print(fake_logreg.get_params())
print(logreg.get_params())

In [ ]:
evaluation = model_selection.cross_val_score(fake_logreg, fX, fY, cv=10, scoring='accuracy')
print(evaluation.mean(), evaluation.std())

In [ ]:
evaluation = model_selection.cross_val_score(logreg, X, Y, cv=10, scoring='accuracy')
print(evaluation.mean(), evaluation.std())

In [ ]:
#########
# MISCHIANDO

y_pred = fake_logreg.predict(np.array(x_test))
y_test = Y[:10000]
print('Acc', accuracy_score(y_pred=y_pred, y_true=y_test))
print('F1', f1_score(y_pred=y_pred, y_true=y_test))

In [ ]:
#########
# MISCHIANDO

y_pred = logreg.predict(np.array(fX[:4000]))
y_test = fY[:4000]


print('Accuracy', accuracy_score(y_pred=y_pred, y_true=y_test))
print('F1', f1_score(y_pred=y_pred, y_true=y_test))

In [ ]:
sum(Y)/len(Y)

In [ ]:
logreg.coef_

In [ ]:
fake_logreg.coef_

In [ ]:
print('fake wiener', 
      statistics.mean(fwiener_total.values()), 
      statistics.mode(fwiener_total.values()), 
      statistics.stdev(fwiener_total.values()),
      statistics.median(fwiener_total.values()))
print('nonfake wiener', 
      statistics.mean(wiener_total.values()), 
      statistics.mode(wiener_total.values()), 
      statistics.stdev(wiener_total.values()),
      statistics.median(wiener_total.values()))

In [ ]:
fpr = 1
tpr = 1
fpr,tpr,_ =  roc_curve(y_test, y_score)

In [ ]:
fake_logreg.fit(fX, fY).decision_function(fX[:4000])

In [ ]:
for the_id in db.fakes.find({}, {'msg_id':1, '_id':0}):
    if db.follows.find({'[on':int(the_id['msg_id'])}).count() > 0:
        print(the_id)
        break

In [ ]:
for el in db.follows.find():
    print (el['[on'], type(el['[on']))
    break

In [ ]:
# average edges
nfoutd = list()
nfnodes = list()
for comp in (nx.weakly_connected_component_subgraphs(nfake_graph)):
    nfoutd.append(len(comp.edges()))
    nfnodes.append(len(comp.nodes()))

In [ ]:
statistics.mean(nfnodes)

In [ ]:
# average edges
foutd = list()
fnodes = list()
for comp in (nx.weakly_connected_component_subgraphs(fake_graph)):
    foutd.append(len(comp.edges()))
    fnodes.append(len(comp.nodes()))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()

In [ ]:
pca.fit(X)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.n_components = 6
X_red = pca.fit_transform(X)
X_red.shape

In [ ]:
evaluation = model_selection.cross_val_score(logreg, X, Y, cv=10, scoring='accuracy')
print(evaluation.mean(), evaluation.std())

In [ ]:
dbscan = DBSCAN(n_jobs=4)
dbscan_result = dbscan.fit_predict(X+fX)

In [ ]:
set(dbscan_result)

In [ ]:
for el in set(dbscan_result):
    print(el, ': ', np.count_nonzero(dbscan_result == int(el)))

In [ ]:
len(dbscan_result)

In [ ]:
kmeans_model = KMeans(n_clusters=20, n_jobs=4)
kmeans_result = kmeans_model.fit_predict(X+fX)

In [ ]:
for cluster,element in zip(kmeans_result, X+fX):
    if cluster == 2:
        print(cluster, ' : ', element)

In [ ]:
for el in set(kmeans_result):
    print(el, ': ', np.count_nonzero(kmeans_result == int(el)))

In [ ]:
less_X = list()
for el in X+fX:
    less_X.append(el[5:])

In [ ]:
dbscan2 = DBSCAN(n_jobs=4)
dbscan_result2 = dbscan2.fit_predict(less_X)

In [ ]:
for el in set(dbscan_result2):
    print(el, ': ', np.count_nonzero(dbscan_result2 == int(el)))

In [ ]:
kmeans_model2 = KMeans(n_clusters=20, n_jobs=4)
kmeans_result2 = kmeans_model.fit_predict(less_X)

In [ ]:
for el in set(kmeans_result2):
    print(el, ': ', np.count_nonzero(kmeans_result2 == int(el)))